In [4]:
pip install numpy scipy torch torchvision torchaudio

  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/54/4b/b0861005f5d4370b3529f31d5e6461c4faf9bbbcbe916480cceebc885aa8/torchvision-0.16.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/2a/36/96895107b44f41cda87bf41d344cc7b7a1a4be8fff9bda1c66da5ab30051/torchaudio-2.1.2-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for pillow!=8.3.*,>=5.3.0 from https://files.pythonhosted.org/packages/0d/a9/1f4c54afaf2e689ba40b7688095bb70be8e84ec206c7dfe156c0645a0d52/pillow-10.2.0-cp38-cp38-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 991.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.1 MB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.2/1.1 MB 1.6 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.1 MB


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import sys
import os
import numpy as np
from scipy.io import wavfile

In [7]:
import numpy as np
import random
import torch
import torch.optim as optim
import argparse
import time
import csv
from scipy.io.wavfile import write

# Preprocessing

This code is ....

### Code

Define all necessary functions

In [30]:
def prepareTrainingData(config):
    in_rate, in_data = wavfile.read(config["input_audio_path"])
    out_rate, out_data = wavfile.read(config["target_audio_path"])
    
    if len(in_data) != len(out_data):
        print("input and target files have different lengths")
        sys.exit()
      
    if len(in_data.shape) > 1 or len(out_data.shape) > 1:
        print("expected mono files")
        sys.exit()

    # Convert PCM16 to FP32
    if in_data.dtype == "int16":
        in_data = in_data / 32767
        print("In data converted from PCM16 to FP32")
    if out_data.dtype == "int16":
        out_data = out_data / 32767
        print("Out data converted from PCM16 to FP32")    

    clean_data = in_data.astype(np.float32).flatten()
    target_data = out_data.astype(np.float32).flatten()

    # Split the data on a twenty percent mod
    in_train, out_train, in_val, out_val = sliceOnMod(clean_data, target_data)

    save_wav(config["output_path"] + "/train/" + config["name"] + "-input.wav", in_train)
    save_wav(config["output_path"] + "/train/" + config["name"] + "-target.wav", out_train)

    save_wav(config["output_path"] + "/test/" + config["name"] + "-input.wav", in_val)
    save_wav(config["output_path"] + "/test/" + config["name"] + "-target.wav", out_val)

    save_wav(config["output_path"] + "/val/" + config["name"] + "-input.wav", in_val)
    save_wav(config["output_path"] + "/val/" + config["name"] + "-target.wav", out_val)


In [31]:
def sliceOnMod(input_data, target_data, mod=5):
    # Split the data on a modulus.

    # Type cast to an integer the modulus
    mod = int(mod)

    # Split the data into 100 pieces
    input_split = np.array_split(input_data, 100)
    target_split = np.array_split(target_data, 100)

    val_input_data = []
    val_target_data = []
    # Traverse the range of the indexes of the input signal reversed and pop every 5th for val
    for i in reversed(range(len(input_split))):
        if i % mod == 0:
            # Store the validation data
            val_input_data.append(input_split[i])
            val_target_data.append(target_split[i])
            # Remove the validation data from training
            input_split.pop(i)
            target_split.pop(i)

    # Flatten val_data down to one dimension and concatenate
    val_input_data = np.concatenate(val_input_data)
    val_target_data = np.concatenate(val_target_data)

    # Concatenate back together
    training_input_data = np.concatenate(input_split)
    training_target_data = np.concatenate(target_split)
    return (training_input_data, training_target_data, val_input_data, val_target_data)

In [32]:
def save_wav(name, data):
    wavfile.write(name, 44100, data.flatten().astype(np.float32))

In [ ]:
def train():
    # Generate name of directory where results will be saved
    save_path = os.path.join(args.save_location, args.device + '-' + args.load_config)

In [35]:
config = {
    "hidden_size": 20,
    "unit_type": "LSTM",
    "loss_fcns": {"ESR": 0.75, "DC": 0.25},
    "pre_filt": "high_pass",
    "device": "acoustic1-pre",
    "file_name": "acoustic1-pre",
    "name": "test",
    "input_audio_path": "Data/val/ht1-input.wav",
    "target_audio_path": "Data/val/ht1-target.wav",
    "output_path": "Data"
}

prepareTrainingData(config)

In data converted from PCM16 to FP32
Out data converted from PCM16 to FP32
